In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from pmdarima import auto_arima

# Suppress tensorflow logging due to un-relevant warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
training_split = 19304
validation_split = 4826
test_split = 2682 

asset_folder = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', 'HeroysundBridge-ML-Assets'))
df = pd.read_parquet(os.path.join(asset_folder, 'silver','combined_data_v01.parquet'))
df.index = pd.to_datetime(df['Date'], format='%Y%m%d%H')

# Naive Prediction (t + 1)

Reference: isbn = {1492078190}, title = {AI and Machine Learning for Coders: A Programmer's Guide to Artificial Intelligence}, author = {Moroney, Laurence}, Page(s): 169-171

In [ ]:
delay = 1

test_df = df[['Point_1_N_mean']][(training_split + validation_split):]
naive_df = df[['Point_1_N_mean']][((training_split + validation_split) - delay):-delay]

plt.figure(figsize=(10, 5))
plt.plot(naive_df.index, df[(training_split + validation_split):]['Point_1_N_mean'], label='Test Dataset')
plt.plot(naive_df.index, naive_df['Point_1_N_mean'], label='Naive Prediction')
plt.xlabel('Date')
plt.ylabel('Point 1 N mean')
plt.legend()
plt.show()

print(f"MSE: {mean_squared_error(test_df['Point_1_N_mean'], naive_df['Point_1_N_mean']).numpy()}")
print(f"MAE: {mean_absolute_error(test_df['Point_1_N_mean'], naive_df['Point_1_N_mean']).numpy()}")

# Sckit-learn linear regression model

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
lin_reg_df = df[(training_split + validation_split):]

model = LinearRegression()

# Split the data into features (X) and target variable (y)
X = lin_reg_df[['Average_Global_Radiation_(1h)', 'PT100_Temperature_mean']]
y = lin_reg_df['Point_1_N_mean']

model.fit(X, y)
predictions = model.predict(X)

plt.figure(figsize=(10, 5))
plt.plot(lin_reg_df.index, y, label='Test Dataset')
plt.plot(lin_reg_df.index, predictions, label='Linear Regression Prediction')
plt.legend()
plt.show()

print(f"MSE: {mean_squared_error(y, predictions)}")
print(f"MAE: {mean_absolute_error(y, predictions)}")

# Moving average

In [ ]:
window = 24

test_df = df[(training_split + validation_split):][['Point_1_N_mean']].dropna()
moving_avg = df[(training_split + validation_split):][['Point_1_N_mean']].rolling(window=window).mean().dropna()

plt.figure(figsize=(10, 5))
plt.plot(test_df.index, test_df['Point_1_N_mean'], label='Test Dataset')
plt.plot(moving_avg.index, moving_avg, label='Moving Average')
plt.xlabel('Index')
plt.ylabel('Point 1 N mean')
plt.legend()
plt.show()

print(f"MSE: {mean_squared_error(test_df['Point_1_N_mean'][window-1:], moving_avg['Point_1_N_mean'].to_numpy())}")
print(f"MAE: {mean_absolute_error(test_df['Point_1_N_mean'][window-1:], moving_avg['Point_1_N_mean'].to_numpy())}")

# Seasonal Decomposition

Reference: https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html#statsmodels.tsa.seasonal.seasonal_decompose

(Copilot-generated description!!!) Statsmodels is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration.

In [ ]:
window = 24

test_df = df[(training_split + validation_split):][['Point_1_N_mean']]
seasonal_df = seasonal_decompose(test_df, model='additive', period=window)
seasonal_df.plot()

# Extract the trend component from the seasonal decomposition
seasional_trend_df = pd.DataFrame(seasonal_df.trend)

# Drop NaN values from both series
test_dropped_df = test_df.dropna()
seasional_trend_dropped_df = seasional_trend_df.dropna()
test_truncated = test_dropped_df.iloc[:len(seasional_trend_dropped_df)]

# Plot the original data, truncated data, and the trend component
plt.figure(figsize=(10, 5))
#plt.plot(test_dropped_df.index, test_dropped_df['Point_1_N_mean'], label='Original Data', linestyle='--')
plt.plot(test_truncated.index, test_truncated['Point_1_N_mean'], label='Truncated Data')
plt.plot(seasional_trend_dropped_df.index, seasional_trend_dropped_df['trend'], label='Trend Component')
plt.xlabel('Index')
plt.ylabel('Point 1 N mean')
plt.legend()
plt.title('Original Data vs. Trend Component vs. Truncated Data')
plt.show()

print(f"MSE: {mean_squared_error(test_truncated['Point_1_N_mean'], seasional_trend_dropped_df['trend'])}")
print(f"MAE: {mean_absolute_error(test_truncated['Point_1_N_mean'], seasional_trend_dropped_df['trend'])}")

# Augmented Dickey-Fuller test + KPSS test

(GitHub Copilot) {
The KPSS and ADF tests should be applied to the dataset that you are using to train your machine learning model. This is because the purpose of these tests is to help you understand the underlying properties of your data so that you can appropriately preprocess the data and select a suitable model. If your training data is non-stationary, the model you train on this data may not perform well on the test data, even if the test data is from the same non-stationary distribution. This is because many machine learning models, especially linear models and time series models, assume that the data is stationary. Applying the tests to the test set would not be as useful, because you do not use the test set to train your model. The test set is used to evaluate the performance of the model that was trained on the training set. However, if the test set is from a different distribution than the training set, this could indicate a problem with your data splitting method. In summary, apply the KPSS and ADF tests to your training data to check for stationarity before training your model. If the tests indicate that your data is non-stationary, you may need to transform your data to make it stationary before training your model.}

Relevant Article: https://medium.com/@tannyasharma21/comparision-study-of-adf-vs-kpss-test-c9d8dec4f62a

Reference: https://www.machinelearningplus.com/time-series/augmented-dickey-fuller-test/

The Augmented Dickey-Fuller Test is used to determine if time-series data is stationary or not. Similar to a t-test, we set a significance level before the test and make conclusions on the hypothesis based on the resulting p-value. "Another point to remember is the ADF test is fundamentally a statistical significance test" - Selva Prabhakaran (https://www.machinelearningplus.com/time-series/augmented-dickey-fuller-test/)


Reference: https://www.machinelearningplus.com/time-series/kpss-test-for-stationarity/

In [ ]:
# (Info!) This cell is heavily influenced by GitHub Copilot

# ADF Test
print('Results of Augmented Dickey-Fuller Test (ADF):')
dftest_adf = adfuller(df[:(validation_split + test_split)]['Point_1_N_mean'], autolag='AIC')  # Choosing data that ML model has seen (Excluding test data)

# Create a Series to hold the ADF test results
adf_output = pd.Series(dftest_adf[0:4], index=['Test Statistic (ADF)', 'p-value (ADF)', '#Lags Used (ADF)', 'Number of Observations Used (ADF)'])

# Add critical values to the Series
for key, value in dftest_adf[4].items():
    adf_output[f'Critical Value ({key}) (ADF)'] = value

print(adf_output)

# Determine if we reject the null hypothesis based on a significance level of 0.05
if adf_output['p-value (ADF)'] < 0.05:
    print("Reject the null hypothesis (H0) - Data is stationary (ADF)")
else:
    print("Fail to reject the null hypothesis (H0) - Data is non-stationary (ADF)")

# KPSS Test
print('\nResults of KPSS Test:')
kpsstest = kpss(df[:(validation_split + test_split)]['Point_1_N_mean'], regression='ct', nlags='auto')

# Create a Series to hold the KPSS test results
kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic (KPSS)', 'p-value (KPSS)', 'Lags Used (KPSS)'])

# Add critical values to the Series
for key, value in kpsstest[3].items():
    kpss_output[f'Critical Value ({key}) (KPSS)'] = value

print(kpss_output)

# Determine if we reject the null hypothesis based on a significance level of 0.05
if kpss_output['p-value (KPSS)'] < 0.05:
    print("Reject the null hypothesis (H0) - Data is non-stationary (KPSS)")
else:
    print("Fail to reject the null hypothesis (H0) - Data is stationary (KPSS)")

In [ ]:
# (Info!) This cell is heavily influenced by GitHub Copilot

# Perform seasonal decomposition
seasonal_df = seasonal_decompose(df[:(validation_split + test_split)]['Point_1_N_mean'], model='additive', period=window)
residuals = df[:(validation_split + test_split)]['Point_1_N_mean'] - seasonal_df.trend  # Extract residuals

# ADF Test on residuals
print('Results of Augmented Dickey-Fuller Test (ADF) on Residuals:')
dftest_adf_resid = adfuller(residuals.dropna(), autolag='AIC')  # Drop NaN values from residuals before testing

# Create a Series to hold the ADF test results for residuals
adf_output_resid = pd.Series(dftest_adf_resid[0:4], index=['Test Statistic (ADF Residuals)', 'p-value (ADF Residuals)', '#Lags Used (ADF Residuals)', 'Number of Observations Used (ADF Residuals)'])

# Add critical values to the Series
for key, value in dftest_adf_resid[4].items():
    adf_output_resid[f'Critical Value ({key}) (ADF Residuals)'] = value

print(adf_output_resid)

# Determine if we reject the null hypothesis based on a significance level of 0.05
if adf_output_resid['p-value (ADF Residuals)'] < 0.05:
    print("Reject the null hypothesis (H0) - Data is stationary (ADF Residuals)")
else:
    print("Fail to reject the null hypothesis (H0) - Data is non-stationary (ADF Residuals)")

# KPSS Test on residuals
print('\nResults of KPSS Test on Residuals:')
kpsstest_resid = kpss(residuals.dropna(), regression='ct', nlags='auto')  # Drop NaN values from residuals before testing

# Create a Series to hold the KPSS test results for residuals
kpss_output_resid = pd.Series(kpsstest_resid[0:3], index=['Test Statistic (KPSS Residuals)', 'p-value (KPSS Residuals)', 'Lags Used (KPSS Residuals)'])

# Add critical values to the Series
for key, value in kpsstest_resid[3].items():
    kpss_output_resid[f'Critical Value ({key}) (KPSS Residuals)'] = value

print(kpss_output_resid)

# Determine if we reject the null hypothesis based on a significance level of 0.05
if kpss_output_resid['p-value (KPSS Residuals)'] < 0.05:
    print("Reject the null hypothesis (H0) - Data is non-stationary (KPSS Residuals)")
else:
    print("Fail to reject the null hypothesis (H0) - Data is stationary (KPSS Residuals)")

# SARIMAX  (Seasonal Autoregressive Integrated Moving-Average with Exogenous Regressors)

References: 

https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html#statsmodels.tsa.statespace.sarimax.SARIMAX

https://towardsdatascience.com/time-series-forecasting-with-arima-sarima-and-sarimax-ee61099e78f6

https://pypi.org/project/pmdarima/

(Copilot-generated description!!!) SARIMAX is an acronym for Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors model. It is a class of models that explains a given time series based on its own past values, that is, its own lags and the lagged forecast errors, so that equation can be used to forecast future values.

In [ ]:
a=15000 # Varaible to shift how big of training+vaildation set we want to use

data = pd.read_parquet(os.path.join(asset_folder, 'silver','combined_data_v01.parquet'))
data.index = pd.to_datetime(data['Date'], format='%Y%m%d%H').dropna()
data = data[['Point_1_N_mean', 'Average_Global_Radiation_(1h)','PT100_Temperature_mean']][a:-test_split]
data['month'] = data.index.month
data['hour'] = data.index.hour
data['day'] = data.index.dayofyear
data.index = pd.to_datetime(data.index)

In [ ]:
model = auto_arima(
    data['Point_1_N_mean'],  # The time series to which the ARIMA model is fit.
    exogenous=data[['month', 'hour', 'day', 'Average_Global_Radiation_(1h)', 'PT100_Temperature_mean']],  # The external variables to include in the model.
    start_p=1,  # This is the order of the AR term.
    start_q=1,  # This is the order of the MA term.
    test='adf',  # The type of unit root test to use in order to determine the order of differencing.
    max_p=3,  # The maximum value of p to try when fitting the model.
    max_q=3,  # The maximum value of q to try when fitting the model.
    m=7,  # The number of periods in each season. This affects the seasonal differencing.
    start_P=0,  # The starting value of P in auto_arima. This is the order of the seasonal AR term.
    seasonal=True,  # Whether to include seasonal differencing in the model.
    d=None,  # The order of first-differencing. If None, the value is automatically determined.
    D=1,  # The order of seasonal differencing.
    trace=True,  # Whether to print status on the fits.
    error_action='ignore',  # If a model cannot be fit, ignore the error and continue.
    suppress_warnings=True,  # If True, do not print warnings.
    stepwise=True,  # If True, use the stepwise algorithm to fit the model.
    maxiter=15  # The maximum number of function evaluations.
)

model.plot_diagnostics(figsize=(15, 12))
plt.show()
model.summary()

Following code is reused from the following code: https://gist.githubusercontent.com/brendanartley/c69185f28e678c7221546a9c43825ec0/raw/d702ee2d1e4ad57e4ea9f92e5fa09a3414b509e8/gistfile1.txt
Reference: https://towardsdatascience.com/time-series-forecasting-with-arima-sarima-and-sarimax-ee61099e78f6

In [ ]:
def forecast(model, periods=test_split): # , d = df[(training_split + validation_split):])
    # Forecast
    forecast, conf_int = model.predict(n_periods=periods, return_conf_int=True)
    
    # Generate date range for forecast
    forecast_index = pd.date_range(start=data.index[-1], periods=periods + 1, freq='H')[1:]
   
    # Define start and end indices for plotting
    lookback_variable = 500*3

    # Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(df.index[(training_split+validation_split)-lookback_variable:-test_split], df['Point_1_N_mean'].iloc[(training_split+validation_split)-lookback_variable:-test_split], label='Training and Validation Dataset', color='black')
    plt.plot(df[-test_split:].index, df['Point_1_N_mean'][-test_split:])
    plt.plot(forecast_index, forecast, label='SARIMAx Forecast')
    plt.fill_between(forecast_index, conf_int[:, 0], conf_int[:, 1], color='k', alpha=0.2) # Between the confidence intervals
    plt.title("SARIMAx Forecast")
    plt.xlabel("Date")
    plt.ylabel("Point_1_N_mean")
    plt.legend()
    plt.show()

    forecast_values = forecast.to_numpy().flatten()
    test_values = test_df.to_numpy().flatten()
   
    print(f"MSE: {mean_squared_error(test_values, forecast_values)}")
    print(f"MAE: {mean_absolute_error(test_values, forecast_values)}")

forecast(model)
